In [1]:
import cv2 as cv
import mediapipe as mp


In [2]:
import numpy as np
def image_overlay_third(img1,img2,location):
    h,w = img1.shape[:2]
    h1,w1 = img2.shape[:2]
    x,y = location
    roi = img1[y:y+h1 , x:x+w1]
    
    gray = cv.cvtColor(img2,cv.COLOR_BGR2GRAY)
    _,mask = cv.threshold(gray,150,255,cv.THRESH_BINARY)
    mask_inv = cv.bitwise_not(mask)
    
    img_bg = cv.bitwise_and(roi,roi,mask=mask)
    img_fg = cv.bitwise_and(img2,img2,mask=mask_inv)
    
    final = cv.add(img_bg,img_fg)
    
    img1[y:y+h1,x:x+w1] = final

    return img1

In [3]:
import time
if __name__ == '__main__':

    pic_no = 0
    cap = cv.VideoCapture(0)
    
    overlay = cv.imread("./my_style.png")
    overlay = cv.resize(overlay ,(140,160))

    overlay1 = cv.imread("./demon.png")
    overlay1 = cv.resize(overlay1 ,(250,270))
    
    overlay2 = cv.imread("./pika.png")
    overlay2 = cv.resize(overlay2 ,(170,180))
    
    mpface = mp.solutions.face_detection
    face = mpface.FaceDetection()

    mpdraw = mp.solutions.drawing_utils
    rel_flag = 1
    filter_no = 0
    while cap.isOpened():
        try:
            
            ret,img = cap.read()
            img = cv.resize(img ,(500,400))

            img3 = cv.cvtColor(img,cv.COLOR_BGR2RGB)
            results = face.process(img3)
            #print(results.detections)





            frame = cv.cvtColor(img,cv.COLOR_BGR2HSV_FULL)
            
            if results.detections:
                for id,detection in enumerate(results.detections):
                #mpdraw.draw_detection(frame,detection)
                    bbox = detection.location_data.relative_bounding_box
                    ih,iw,ic = img3.shape
                    #cv.rectangle(img,(int(bbox.xmin *iw)-10,int(bbox.ymin *ih)-40),(int((bbox.xmin + bbox.width)* iw),int((bbox.ymin + bbox.height)*ih)),(255,0,255),2)
                    roi = img[int(bbox.ymin *ih)-40:int((bbox.ymin + bbox.height)*ih),int(bbox.xmin *iw)-10:int((bbox.xmin + bbox.width)* iw)]
                    if filter_no >= 3:
                        filter_no = 0
                    
                    if filter_no == 0:
                        after_overlay=image_overlay_third(img,overlay,location = (int(bbox.xmin *iw)-10,int(bbox.ymin *ih)-80))
                    elif filter_no == 1:
                        after_overlay=image_overlay_third(img,overlay1,location = (int(bbox.xmin *iw)-70,int(bbox.ymin *ih)-160))
                    elif filter_no == 2:
                        after_overlay=image_overlay_third(img,overlay2,location = (int(bbox.xmin *iw)-25,int(bbox.ymin *ih)-50))  
                    if rel_flag == 1:
                        
                       ''' relative = list(roi.shape)
                    if list(roi.shape)[0]>relative[0]:
                        overlay = cv.resize(overlay,(list(overlay.shape)[0] +1,list(overlay.shape)[1] +1))
                        overlay1 = cv.resize(overlay1,(list(overlay1.shape)[0] +1,list(overlay1.shape)[1] +1))
                        overlay2 = cv.resize(overlay2,(list(overlay2.shape)[0] +1,list(overlay2.shape)[1] +1))
                    if list(roi.shape)[0]<relative[0]:
                        overlay = cv.resize(overlay,(list(overlay.shape)[0] -1,list(overlay.shape)[1] -1))
                        overlay1 = cv.resize(overlay1,(list(overlay1.shape)[0] -1,list(overlay1.shape)[1] -1))
                        overlay2 = cv.resize(overlay2,(list(overlay2.shape)[0] -1,list(overlay2.shape)[1] -1))
                        '''
                    

            cv.imshow("img",after_overlay)
        except Exception as e:
            
            pass
        
        pressed_key = cv.waitKey(1)
        if pressed_key == 27 :
            break
        if pressed_key == ord("n") :
            filter_no += 1
        
        if pressed_key == ord("w") :
            cv.imwrite(f"pic{pic_no}.jpg",after_overlay)
            pic_no += 1
            print(pic_no)
            blank = np.ones((500,400,1),dtype="uint8")
            cv.imshow("img",blank)
            time.sleep(1)
        rel_flag = -1   

    cap.release()
    cv.destroyAllWindows()


1
2
3
4
5
6
7
